In [1]:
import pandas as pd
import numpy as np

# 1. CARICAMENTO DATI
# Carichiamo il dataset originale
try:
    df = pd.read_csv('data.csv')
    print(f"Dataset caricato con successo: {df.shape[0]} righe, {df.shape[1]} colonne.")
except FileNotFoundError:
    print("Errore: Il file 'data.csv' non è stato trovato. Assicurati che sia nella stessa cartella.")

# ---------------------------------------------------------

# 2. SELEZIONE COLONNE (PULIZIA)
# Teniamo solo le colonne utili per la visualizzazione e l'analisi
cols_to_keep = [
    'Conference', 
    'Year', 
    'Title', 
    'AuthorNames-Deduped', # Nomi puliti
    'AuthorAffiliation',   # Serve per trovare il paese
    'AminerCitationCount', # Citazioni
    'DOI'
]

# Creiamo un nuovo dataframe ridotto
df_clean = df[cols_to_keep].copy()

# Rinominiamo le colonne per renderle più leggibili nel codice JS
df_clean.columns = ['Conference', 'Year', 'Title', 'Authors', 'Affiliation', 'Citations', 'DOI']

# Gestione valori mancanti (NaN) per le citazioni e affiliazioni
df_clean['Citations'] = df_clean['Citations'].fillna(0).astype(int)
df_clean['Affiliation'] = df_clean['Affiliation'].fillna("Unknown")

# ---------------------------------------------------------

# 3. ESTRAZIONE DEL PAESE (COUNTRY)
# Funzione per estrarre la nazione dalla stringa dell'affiliazione

def get_country(affiliation_string):
    if not isinstance(affiliation_string, str) or affiliation_string == "Unknown":
        return "Other"
    
    # 1. Prendiamo solo la PRIMA affiliazione (prima del punto e virgola)
    # Esempio: "Stanford Univ, USA; Harvard, USA" -> "Stanford Univ, USA"
    first_affiliation = affiliation_string.split(';')[0].strip()
    
    # 2. Dividiamo per virgola per trovare l'ultima parte (spesso è il paese)
    parts = first_affiliation.split(',')
    last_part = parts[-1].strip().upper() # Es: " USA" -> "USA"
    
    # 3. Mappatura manuale per casi comuni e pulizia
    # Normalizziamo i nomi più frequenti per avere gruppi puliti nel grafico
    if last_part in ['USA', 'UNITED STATES', 'US', 'CA', 'MA', 'NY']: return "North America"
    if 'USA' in first_affiliation.upper(): return "North America"
    if 'CANADA' in first_affiliation.upper(): return "North America"
    
    if last_part in ['UK', 'UNITED KINGDOM', 'ENGLAND']: return "Europe"
    if last_part in ['GERMANY', 'DEUTSCHLAND']: return "Europe"
    if last_part in ['FRANCE']: return "Europe"
    if last_part in ['ITALY', 'ITALIA']: return "Europe"
    if last_part in ['SWITZERLAND', 'ZURICH']: return "Europe"
    if last_part in ['AUSTRIA']: return "Europe"
    if last_part in ['NETHERLANDS', 'THE NETHERLANDS']: return "Europe"
    
    if last_part in ['CHINA', 'P.R. CHINA']: return "Asia"
    if last_part in ['JAPAN']: return "Asia"
    if last_part in ['KOREA', 'SOUTH KOREA']: return "Asia"
    if last_part in ['SINGAPORE']: return "Asia"
    
    if last_part in ['AUSTRALIA']: return "Asia/Oceania"
    
    # Se non riusciamo a categorizzare facilmente, proviamo a cercare parole chiave nel testo
    txt = first_affiliation.upper()
    if "USA" in txt or "UNIV" in txt and "CA" in parts: return "North America" # Euristica per USA
    if "GERMANY" in txt or "FRANCE" in txt or "UK" in txt: return "Europe"
    if "CHINA" in txt or "JAPAN" in txt: return "Asia"
    
    return "Other" # Per tutto il resto

# Applichiamo la funzione
print("Estrazione Paesi in corso...")
df_clean['Region'] = df_clean['Affiliation'].apply(get_country)

# ---------------------------------------------------------

# 4. SALVATAGGIO
# Salviamo il file pulito che userà il sito web
df_clean.to_csv('data_processed.csv', index=False)

print("\n--- OPERAZIONE COMPLETATA ---")
print("File 'data_processed.csv' creato.")
print("\nPrime 5 righe del nuovo dataset:")
display(df_clean.head())

# Controllo distribuzione regioni
print("\nDistribuzione Regioni:")
print(df_clean['Region'].value_counts())

Dataset caricato con successo: 3877 righe, 20 colonne.
Estrazione Paesi in corso...

--- OPERAZIONE COMPLETATA ---
File 'data_processed.csv' creato.

Prime 5 righe del nuovo dataset:


,Conference,Year,Title,Authors,Affiliation,Citations,DOI,Region
0,InfoVis,2011,D³ Data-Driven Documents,Michael Bostock;Vadim Ogievetsky;Jeffrey Heer,"Computer Science Department, Stanford Universi...",3795,10.1109/tvcg.2011.185,North America
1,InfoVis,2014,UpSet: Visualization of Intersecting Sets,Alexander Lex;Nils Gehlenborg;Hendrik Strobelt...,Hendrik Strobelt and Hanspeter Pfister are wit...,1243,10.1109/tvcg.2014.2346248,Other
2,InfoVis,2010,Narrative Visualization: Telling Stories with ...,Edward Segel;Jeffrey Heer,"University of Stanford, Stanford, CA, USA;Univ...",1408,10.1109/tvcg.2010.179,North America
3,InfoVis,2006,Hierarchical Edge Bundles: Visualization of Ad...,Danny Holten,"Technische Universiteit Eindhoven, Netherlands",1395,10.1109/tvcg.2006.147,Europe
4,InfoVis,2007,Toward a Deeper Understanding of the Role of I...,Ji Soo Yi;Youn ah Kang;John T. Stasko;Julie A....,"Health Systems Institute, Georgia Institute of...",1149,10.1109/tvcg.2007.70515,North America



Distribuzione Regioni:
Region
North America    1773
Other            1051
Europe            805
Asia              225
Asia/Oceania       23
Name: count, dtype: int64
